# The Pst class

The `pst_handler` module contains the `Pst` class for dealing with pest control files.  It relies heavily on `pandas` to deal with tabular sections, such as parameters, observations, and prior information.  

In [1]:
from __future__ import print_function
import os
import numpy as np
import pyemu
from pyemu import Pst

flopy is installed in /Users/jwhite/Dev/flopy/flopy


We need to pass the name of a pest control file to instantiate:

In [2]:
pst_name = os.path.join("henry","pest.pst")
p = Pst(pst_name)

All of the relevant parts of the pest control file are attributes of the `pst` class with the same name:



In [3]:
p.parameter_data.head()

,parnme,partrans,parchglim,parval1,parlbnd,parubnd,pargp,scale,offset,dercom,extra
parnme,,,,,,,,,,,
global_k,global_k,log,factor,200.0,150.00,250.00,m,1.0,0.0,1,NaN
mult1,mult1,log,factor,1.0,0.75,1.25,m,1.0,0.0,1,NaN
mult2,mult2,log,factor,1.0,0.50,2.00,m,1.0,0.0,1,NaN
kr01c01,kr01c01,log,factor,1.0,0.10,10.00,p,1.0,0.0,1,NaN
kr01c02,kr01c02,log,factor,1.0,0.10,10.00,p,1.0,0.0,1,NaN


In [4]:
p.observation_data.head()

,obsnme,obsval,weight,obgnme,extra
obsnme,,,,,
h_obs01_1,h_obs01_1,0.051396,152.1458,head,NaN
h_obs01_2,h_obs01_2,0.022156,0.0000,head,NaN
h_obs02_1,h_obs02_1,0.046879,152.1458,head,NaN
h_obs02_2,h_obs02_2,0.020853,0.0000,head,NaN
h_obs03_1,h_obs03_1,0.036584,152.1458,head,NaN


In [5]:
p.prior_information.head()

,pilbl,equation,weight,obgnme
pilbl,,,,
mult1,mult1,1.0 * log(mult1) = 0.000000,1.0,regul_m
kr01c01,kr01c01,1.0 * log(kr01c01) = 0.0,1.0,regul_p
kr01c02,kr01c02,1.0 * log(kr01c02) = 0.0,1.0,regul_p
kr01c03,kr01c03,1.0 * log(kr01c03) = 0.0,1.0,regul_p
kr01c04,kr01c04,1.0 * log(kr01c04) = 0.0,1.0,regul_p


A residual file (`.rei` or `res`) can also be passed to the `resfile` argument at instantiation to enable some simple residual analysis and weight adjustments.  If the residual file is in the same directory as the pest control file and has the same base name, it will be accessed automatically:


In [6]:
p.res

,name,group,measured,modelled,residual,weight
name,,,,,,
h_obs01_1,h_obs01_1,head,0.051396,0.080402,-0.029006,152.1458
h_obs01_2,h_obs01_2,head,0.022156,0.036898,-0.014742,0.0000
h_obs02_1,h_obs02_1,head,0.046879,0.069121,-0.022241,152.1458
h_obs02_2,h_obs02_2,head,0.020853,0.034311,-0.013458,0.0000
h_obs03_1,h_obs03_1,head,0.036584,0.057722,-0.021138,152.1458
h_obs03_2,h_obs03_2,head,0.019502,0.031408,-0.011905,0.0000
h_obs04_1,h_obs04_1,head,0.027542,0.045770,-0.018229,152.1458
h_obs04_2,h_obs04_2,head,0.016946,0.027337,-0.010391,0.0000
h_obs05_1,h_obs05_1,head,0.026382,0.031442,-0.005060,152.1458


The `pst` class has some `@decorated` convience methods related to the residuals:

In [7]:
print(p.phi,p.phi_components)

1855.6874378297973 {'conc': 197.0582209611553, 'head': 1658.629216868642, 'regul_m': 0.0, 'regul_p': 0.0}


Some additional `@decorated` convience methods:

In [8]:
print(p.npar,p.nobs,p.nprior)

603 75 601


In [9]:
print(p.par_groups,p.obs_groups)

['m', 'p'] ['head', 'conc']


In [10]:
print(type(p.par_names)) # all parameter names
print(type(p.adj_par_names)) # adjustable parameter names
print(type(p.obs_names)) # all observation names
print(type(p.nnz_obs_names)) # non-zero weight observations

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>


The "control_data" section of the pest control file is accessible in the `Pst.control_data` attribute:

In [11]:
print('jacupdate = {0}'.format(p.control_data.jacupdate))
print('numlam = {0}'.format(p.control_data.numlam))
p.control_data.numlam = 100
print('numlam has been changed to --> {0}'.format(p.control_data.numlam))

jacupdate = 999
numlam = 10
numlam has been changed to --> 100


The `Pst` class also exposes a method to get a new `Pst` instance with a subset of parameters and or obseravtions.  Note this method does not propogate prior information to the new instance:

In [12]:
pnew = p.get(p.par_names[:10],p.obs_names[-10:])
print(pnew.prior_information)

           pilbl                     equation  weight   obgnme      names
pilbl                                                                    
mult1      mult1  1.0 * log(mult1) = 0.000000     1.0  regul_m    [mult1]
kr01c01  kr01c01     1.0 * log(kr01c01) = 0.0     1.0  regul_p  [kr01c01]
kr01c02  kr01c02     1.0 * log(kr01c02) = 0.0     1.0  regul_p  [kr01c02]
kr01c03  kr01c03     1.0 * log(kr01c03) = 0.0     1.0  regul_p  [kr01c03]
kr01c04  kr01c04     1.0 * log(kr01c04) = 0.0     1.0  regul_p  [kr01c04]
kr01c05  kr01c05     1.0 * log(kr01c05) = 0.0     1.0  regul_p  [kr01c05]
kr01c06  kr01c06     1.0 * log(kr01c06) = 0.0     1.0  regul_p  [kr01c06]
kr01c07  kr01c07     1.0 * log(kr01c07) = 0.0     1.0  regul_p  [kr01c07]


You can also write a pest control file with altered parameters, observations, and/or prior information:

In [13]:
pnew.write("test.pst")

noptmax:-1, npar_adj:10, nnz_obs:3


//miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
//miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Some other methods in `Pst` include:

In [14]:
# add preferred value regularization with weights proportional to parameter bounds
pyemu.utils.helpers.zero_order_tikhonov(pnew)
pnew.prior_information

,pilbl,equation,obgnme,weight
pilbl,,,,
global_k,global_k,1.0 * log(global_k) = 2.301030E+00,regulm,4.507576
mult1,mult1,1.0 * log(mult1) = 0.000000E+00,regulm,4.507576
mult2,mult2,1.0 * log(mult2) = 0.000000E+00,regulm,1.660964
kr01c01,kr01c01,1.0 * log(kr01c01) = 0.000000E+00,regulp,0.500000
kr01c02,kr01c02,1.0 * log(kr01c02) = 0.000000E+00,regulp,0.500000
kr01c03,kr01c03,1.0 * log(kr01c03) = 0.000000E+00,regulp,0.500000
kr01c04,kr01c04,1.0 * log(kr01c04) = 0.000000E+00,regulp,0.500000
kr01c05,kr01c05,1.0 * log(kr01c05) = 0.000000E+00,regulp,0.500000
kr01c06,kr01c06,1.0 * log(kr01c06) = 0.000000E+00,regulp,0.500000


In [15]:
# add preferred value regularization with unity weights
pyemu.utils.helpers.zero_order_tikhonov(pnew,parbounds=False)
pnew.prior_information

,pilbl,equation,obgnme,weight
0,global_k,1.0 * log(global_k) = 2.301030E+00,regulm,1.0
1,mult1,1.0 * log(mult1) = 0.000000E+00,regulm,1.0
2,mult2,1.0 * log(mult2) = 0.000000E+00,regulm,1.0
3,kr01c01,1.0 * log(kr01c01) = 0.000000E+00,regulp,1.0
4,kr01c02,1.0 * log(kr01c02) = 0.000000E+00,regulp,1.0
5,kr01c03,1.0 * log(kr01c03) = 0.000000E+00,regulp,1.0
6,kr01c04,1.0 * log(kr01c04) = 0.000000E+00,regulp,1.0
7,kr01c05,1.0 * log(kr01c05) = 0.000000E+00,regulp,1.0
8,kr01c06,1.0 * log(kr01c06) = 0.000000E+00,regulp,1.0
9,kr01c07,1.0 * log(kr01c07) = 0.000000E+00,regulp,1.0


Some more `res` functionality

In [17]:
# adjust observation weights to account for residual phi components
#pnew = p.get()
print(p.phi, p.nnz_obs, p.phi_components)
p.adjust_weights_discrepancy()
print(p.phi, p.nnz_obs, p.phi_components)

1855.6874378297973 36 {'conc': 197.0582209611553, 'head': 1658.629216868642}
24.94998746710643 36 {'conc': 4.357213275308137, 'head': 20.59277419179829}


adjust observation weights by an arbitrary amount by groups:

In [18]:
print(p.phi, p.nnz_obs, p.phi_components)
grp_dict = {"head":100}
p.adjust_weights(obsgrp_dict=grp_dict)
print(p.phi, p.nnz_obs, p.phi_components)

24.94998746710643 36 {'conc': 4.357213275308137, 'head': 20.59277419179829}
104.35721327530817 36 {'conc': 4.357213275308137, 'head': 100.00000000000003}


adjust observation weights by an arbitrary amount by individual observations:

In [19]:
print(p.phi, p.nnz_obs, p.phi_components)
obs_dict = {"h_obs01_1":25}
p.adjust_weights(obs_dict=obs_dict)
print(p.phi, p.nnz_obs, p.phi_components)

104.35721327530817 36 {'conc': 4.357213275308137, 'head': 100.00000000000003}
124.50114099147628 36 {'conc': 4.357213275308137, 'head': 120.14392771616814}


setup weights inversely proportional to the observation values

In [21]:
p.adjust_weights_discrepancy()
print(p.phi, p.nnz_obs, p.phi_components)
p.proportional_weights(fraction_stdev=0.1,wmax=20.0)
print(p.phi, p.nnz_obs, p.phi_components)

25.357213275308137 36 {'conc': 4.357213275308137, 'head': 21.0}
222.9699656215899 36 {'conc': 194.30909581461378, 'head': 28.660869806976095}
